<a href="https://colab.research.google.com/github/GustMont/OtimizarModeloPreTreinado/blob/main/Ponderada_Otimiza%C3%A7%C3%A3o_Semana3_GustavoMonteiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Otimização de Modelo Pré-Treinado para Detecção de Fraudes em Cartões de Crédito


Aluno: Gustavo Monteiro

##Bibliotecas Utilizadas

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import numpy as np

## Carregamento e Divisão para Treino e Teste

In [7]:
data = pd.read_csv('creditcard.csv')
data_sample = data.sample(frac=0.1, random_state=42)

X = data_sample.drop('Class', axis=1)
y = data_sample['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

etapa onde os dados são carregados do arquivo creditcard.csv e uma amostra de 10% é selecionada para facilitar o treinamento. Os dados são então separados em variáveis de entrada (X) e alvo (y), e divididos em conjuntos de treinamento e teste, com 30% dos dados reservados para avaliação.

##Criação do Modelo Original

In [8]:
def create_model(optimizer='adam', init_mode='uniform'):
    model = Sequential()
    model.add(tf.keras.Input(shape=(X_train.shape[1],)))
    model.add(Dense(16, activation='relu', kernel_initializer=init_mode))
    model.add(Dense(8, activation='relu', kernel_initializer=init_mode))
    model.add(Dense(1, activation='sigmoid', kernel_initializer=init_mode))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [9]:
model = create_model()

model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

y_pred = model.predict(X_test).ravel()
y_pred_classes = (y_pred > 0.5).astype(int)

Epoch 1/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9982 - loss: 0.0419
Epoch 2/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9983 - loss: 0.0340
Epoch 3/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9978 - loss: 0.0411
Epoch 4/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9976 - loss: 0.0491
Epoch 5/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9988 - loss: 0.0242
Epoch 6/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9983 - loss: 0.0288
Epoch 7/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9979 - loss: 0.0305
Epoch 8/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9982 - loss: 0.0259
Epoch 9/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9981 - loss: 0.0249
Epoch 10/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9980 - loss: 0.0261
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


Foi criado um modelo de rede neural sequencial para a tarefa de classificação binária. O modelo possui três camadas densas: a primeira com 16 neurônios, a segunda com 8 neurônios e a terceira com um neurônio, que usa a função de ativação sigmoid para produzir a saída.

O treinamento apresentou uma alta precisão de até 99.88%. Contudo, um dos principais problemas observados foi a possibilidade de overfitting. Overfitting

### Avaliação do modelo original

In [10]:
accuracy = accuracy_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)
auc_roc = roc_auc_score(y_test, y_pred)

print(f'Original -> Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}, AUC-ROC: {auc_roc:.4f}')

Original -> Accuracy: 0.9992, Recall: 0.0000, F1-Score: 0.0000, AUC-ROC: 0.7020


A AUC-ROC, está em um nível razoável, mas ainda assim indica que o modelo pode não estar performando de maneira ideal para identificar a classe positiva.

 diferença entre a alta precisão e as baixas métricas de recall e F1-Score podem indicar overfitting, onde o modelo pode estar tendendo a prever predominantemente a classe negativa devido à sua predominância no conjunto de dados.

##Modelagem com Hiperparâmetros

In [11]:
batch_sizes = [16, 32, 64]
epochs = [10, 50, 100]
optimizers = ['adam', 'rmsprop']
init_modes = ['uniform', 'he_uniform', 'glorot_uniform']

best_accuracy = 0
best_params = {}

n_combinations = 10

for i in range(n_combinations):
    batch_size = random.choice(batch_sizes)
    epoch = random.choice(epochs)
    optimizer = random.choice(optimizers)
    init_mode = random.choice(init_modes)

    print(f'Configuração {i+1}/{n_combinations} - Batch Size: {batch_size}, Epochs: {epoch}, Optimizer: {optimizer}, Init Mode: {init_mode}')

    model = create_model(optimizer=optimizer, init_mode=init_mode)
    history = model.fit(X_train, y_train, epochs=epoch, batch_size=batch_size, verbose=0)

    y_pred = model.predict(X_test).ravel()
    y_pred_classes = (y_pred > 0.5).astype(int)

    accuracy = accuracy_score(y_test, y_pred_classes)
    recall = recall_score(y_test, y_pred_classes)
    f1 = f1_score(y_test, y_pred_classes)
    auc_roc = roc_auc_score(y_test, y_pred)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {
            'batch_size': batch_size,
            'epochs': epoch,
            'optimizer': optimizer,
            'init_mode': init_mode,
            'recall': recall,
            'f1': f1,
            'auc_roc': auc_roc
        }

    print(f'Configuração {i+1}/{n_combinations} - Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}, AUC-ROC: {auc_roc:.4f}')

Configuração 1/10 - Batch Size: 32, Epochs: 100, Optimizer: rmsprop, Init Mode: glorot_uniform
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Configuração 1/10 - Accuracy: 0.9980, Recall: 0.2857, F1-Score: 0.1905, AUC-ROC: 0.7044
Configuração 2/10 - Batch Size: 32, Epochs: 50, Optimizer: rmsprop, Init Mode: uniform
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Configuração 2/10 - Accuracy: 0.9992, Recall: 0.0000, F1-Score: 0.0000, AUC-ROC: 0.6310
Configuração 3/10 - Batch Size: 16, Epochs: 50, Optimizer: rmsprop, Init Mode: he_uniform
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Configuração 3/10 - Accuracy: 0.9980, Recall: 0.2857, F1-Score: 0.1905, AUC-ROC: 0.6324
Configuração 4/10 - Batch Size: 32, Epochs: 50, Optimizer: adam, Init Mode: he_uniform
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Configuração 4/10 - Accuracy: 0.9977, Recall: 0.4286, F1-Score: 0.2308, AUC-ROC: 0.7060
Configuração 5/10 - Batch Size: 16, Epochs: 100, Optimizer: rmsprop, Init Mode: glorot_uniform
268/268 ━━━━━━━━━━━━━━━━━━━━ 0

Na etapa de modelagem com hiperparâmetros, a abordagem envolveu a variação de quatro hiperparâmetros principais: tamanho do lote (batch size), número de épocas (epochs), otimizador e modo de inicialização dos pesos. Foram testadas 10 combinações aleatórias desses parâmetros, escolhidas dentre as opções especificadas.

Configurações com alta precisão, mas baixo recall e F1-Score, como a Configuração 2/10 e 7/10, indicam que o modelo pode estar tendo dificuldades para detectar instâncias da classe minoritária, apesar de uma precisão geral elevada.

A Configuração 5/10 teve a maior precisão (accuracy) de 0.9984, além de um recall de 0.5714 e um F1-Score de 0.3636, refletindo um desempenho geral interessante em termos de detecção e balanceamento das classes. Outras configurações, como a 4/10 e a 9/10, também mostraram bom desempenho, especialmente em termos de recall e AUC-ROC, mostrando que a escolha dos parâmetros pode afetar significativamente a capacidade do modelo de identificar a classe minoritária e melhorar a capacidade de generalização.





##Resultado Final

In [12]:
print(f'\nMelhor Accuracy: {best_accuracy:.4f} com parâmetros: {best_params}')


Melhor Accuracy: 0.9992 com parâmetros: {'batch_size': 32, 'epochs': 50, 'optimizer': 'rmsprop', 'init_mode': 'uniform', 'recall': 0.0, 'f1': 0.0, 'auc_roc': 0.630994210755279}


O melhor desempenho foi alcançado com a seguinte configuração: tamanho de lote de 32, 50 épocas, o otimizador rmsprop, e o modo de inicialização uniform. Esta configuração resultou na maior precisão (accuracy) de 0.9992. Mas as métricas de recall, F1-Score e AUC-ROC para esta configuração indicam desafios na detecção da classe minoritária, com recall e F1-Score ambos iguais a 0, e AUC-ROC de ~0.6310.

##Discussão e Conclusão

Esses resultados indicam que, apesar de uma alta precisão geral, o modelo não está conseguindo detectar eficazmente a classe positiva, refletindo possíveis problemas de desequilíbrio de classe e uma limitação na capacidade do modelo de identificar fraudes. Os ajustes de hiperparâmetros puderam melhorar o resultado obtido no teste do modelo original, mas mesmo assim não foram suficientes para superar o desequilíbrio entre as classes. Esse desempenho destaca a necessidade de técnicas adicionais, como balanceamento de dados ou refinamento mais profundo dos hiperparâmetros, para melhorar a eficácia na detecção da classe minoritária.

##Desafios Enfrentados

A indisponibilidade da biblioteca KerasClassifier dificultou a aplicação de técnicas automáticas de ajuste de hiperparâmetros, como grid search e random search. Sem o KerasClassifier, a otimização dos hiperparâmetros teve que ser realizada manualmente, o que aumentou a complexidade e o tempo necessário para a implementação.

Além disso, o Google Colab, que foi utilizado para treinar o modelo, apresentou limitações de recursos computacionais que agravaram o problema. O processamento lento impactou negativamente a eficiência do treinamento e a capacidade de explorar diferentes configurações do modelo, prejudicando o avanço da implementação.

Essa combinação de fatores não só dificultou o avanço na implementação do modelo, mas também levantou a necessidade de considerar soluções alternativas para lidar com grandes volumes de dados e otimizar o tempo de processamento.

## Próximos Passos

É recomendável explorar técnicas de balanceamento de dados, como oversampling e undersampling, para corrigir o desequilíbrio entre as classes e melhorar a capacidade do modelo de identificar a classe positiva, além de revisar e melhorar o pré-processamento dos dados, incluindo normalização, padronização e seleção de características.

Também se faz necessário explorar diferentes arquiteturas de rede neural que possam lidar com grandes volumes de dados, visto que o tempo para processar os dados atuais foi bastante elevado.